In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import sys
import datetime
import math

ROOT_DIR = '../'
sys.path.insert(1, '../production_code/')
from constants import *

In [2]:
area_type = 'region'

df_train = pd.read_csv(ROOT_DIR + ('_' + area_type +'.').join(TESTING_DATA_DIR.split('.')))
df_test = pd.read_csv(ROOT_DIR + ('_' + area_type +'.').join(TESTING_DATA_DIR.split('.')))

In [3]:
df_train

,Day of the Week,Part of Day,Region,Sky,Ambulance,Police
0,Friday,Afternoon,EASTERN REGION,Clear,0.074784,0.064678
1,Friday,Afternoon,EASTERN REGION,Not clear,0.102159,0.093645
2,Friday,Afternoon,METROPOLITAN NORTH WEST REGION,Clear,0.467904,0.388067
3,Friday,Afternoon,METROPOLITAN NORTH WEST REGION,Not clear,0.289449,0.255397
4,Friday,Afternoon,METROPOLITAN SOUTH EAST REGION,Clear,0.519444,0.451734
...,...,...,...,...,...,...
381,Wednesday,Night,NORTHERN REGION,Clear,0.011411,0.007607
382,Wednesday,Night,SOUTH WESTERN REGION,Clear,0.028528,0.028528
383,Wednesday,Night,SOUTH WESTERN REGION,Not clear,0.022866,0.022866
384,Wednesday,Night,WESTERN REGION,Clear,0.017117,0.017117


# basic machine learning model

In [4]:


# # Encode categorical features
# le = LabelEncoder()
# df_train["day"] = le.fit_transform(df_train["day"])
# df_test["day"] = le.transform(df_test["day"])
# le = LabelEncoder()
# df_train["hour_bin"] = le.fit_transform(df_train["hour_bin"])
# df_test["hour_bin"] = le.transform(df_test["hour_bin"])
# le = LabelEncoder()
# df_train["region"] = le.fit_transform(df_train["region"])
# df_test["region"] = le.transform(df_test["region"])
# le = LabelEncoder()
# df_train["sky"] = le.fit_transform(df_train["sky"])
# df_test["sky"] = le.transform(df_test["sky"])


# One hot encoding
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)


df_train[["Ambulance", "Police"]] = df_train[["Ambulance", "Police"]].applymap(lambda x: math.log(x) if x > 0 else pd.NA)
df_test[["Ambulance", "Police"]] = df_test[["Ambulance", "Police"]].applymap(lambda x: math.log(x) if x > 0 else pd.NA)

df_train = df_train.dropna()
df_test = df_test.dropna()

# Split data into features and labels
x_train = df_train.drop(["Ambulance", "Police"], axis=1)
y_train = df_train[["Ambulance", "Police"]]
x_test = df_test.drop(["Ambulance", "Police"], axis=1)
y_test = df_test[["Ambulance", "Police"]]


In [5]:
# Import TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
import numpy as np


# Print the shape and type of the data
print(x_train.shape) # (60000, 28, 28)
print(y_train.shape) # (60000,)
print(x_test.shape) # (10000, 28, 28)
print(y_test.shape) # (10000,)


2023-08-27 18:59:42.995594: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(386, 20)
(386, 2)
(386, 20)
(386, 2)


In [6]:
x_train

,Day of the Week_Friday,Day of the Week_Monday,Day of the Week_Saturday,Day of the Week_Sunday,Day of the Week_Thursday,Day of the Week_Tuesday,Day of the Week_Wednesday,Part of Day_Afternoon,Part of Day_Evening,Part of Day_Morning,Part of Day_Night,Region_EASTERN REGION,Region_METROPOLITAN NORTH WEST REGION,Region_METROPOLITAN SOUTH EAST REGION,Region_NORTH EASTERN REGION,Region_NORTHERN REGION,Region_SOUTH WESTERN REGION,Region_WESTERN REGION,Sky_Clear,Sky_Not clear
0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
3,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
4,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
382,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0
383,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1
384,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,1,0


In [7]:
# y_train = y_train.applymap(lambda x: math.log(x*100))
# y_test = y_test.applymap(lambda x: math.log(x*100))


In [15]:

# Create a model
model = keras.Sequential([
  keras.layers.Dense(20, activation='relu'),
  keras.layers.Dense(100, activation='relu'),
  keras.layers.Dense(10, activation='relu'),
  keras.layers.Dense(2, activation='linear'),
  # keras.layers.Dense(5, activation='softmax')
])


model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error', 'accuracy'])

model.fit(x_train, y_train, batch_size = 10, epochs=100)



Epoch 1/100


39/39 [==============================] - 1s 9ms/step - loss: 4.0053 - mean_squared_error: 4.0053 - accuracy: 0.0155
Epoch 2/100
39/39 [==============================] - 0s 10ms/step - loss: 1.4956 - mean_squared_error: 1.4956 - accuracy: 0.0026
Epoch 3/100
39/39 [==============================] - 0s 9ms/step - loss: 0.3828 - mean_squared_error: 0.3828 - accuracy: 0.5984
Epoch 4/100
39/39 [==============================] - 0s 9ms/step - loss: 0.2917 - mean_squared_error: 0.2917 - accuracy: 0.8135
Epoch 5/100
39/39 [==============================] - 0s 9ms/step - loss: 0.2493 - mean_squared_error: 0.2493 - accuracy: 0.8187
Epoch 6/100
39/39 [==============================] - 0s 9ms/step - loss: 0.2229 - mean_squared_error: 0.2229 - accuracy: 0.8782
Epoch 7/100
39/39 [==============================] - 0s 10ms/step - loss: 0.2034 - mean_squared_error: 0.2034 - accuracy: 0.8912
Epoch 8/100
39/39 [==============================] - 0s 9ms/step - loss: 0.2029 - mean_squared_error: 0.2029 - acc

In [16]:
import math 


# Evaluate the model
model.evaluate(x_test, y_test)

y_pred = pd.DataFrame(model.predict(x_test), columns = ['Predicted Police','Predicted Ambulance'])
y_comb = pd.concat(
            [y_pred, y_test], 
            axis = 1
        ).applymap(math.exp)
y_comb = y_comb.rename(
    columns = {
        'Ambulance': 'True Ambulance', 
        'Police': 'True Police'
    })

y_comb = pd\
    .wide_to_long(
        y_comb.reset_index(), 
        stubnames=['Predicted', 'True'], 
        i='index', 
        j='Emergency Type', 
        sep=' ', 
        suffix='\w+')\
    .reset_index()\
    .drop('index', axis = 1)

fig = px.scatter(
    y_comb,
    x = 'True',
    y = 'Predicted',
    color = 'Emergency Type', symbol = 'Emergency Type'
    )

fig.show()

13/13 [==============================] - 1s 4ms/step
